In [1]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
# Припустимо, у вас є список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# 1. Завантажуємо модель BERT
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(LABEL_COLUMNS))

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 2. Компіляція моделі
bert_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [4]:
# 3. Завантаження токенізованих даних
# Передбачається, що дані вже містять токенізовані вектори input_ids та attention_mask
# Ваша таблиця повинна мати колонки 'input_ids', 'attention_masks' і мітки класів

data_path = '/kaggle/input/train-data/train_data.csv'
df = pd.read_csv(data_path)

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)

# Перетворюємо мітки на категорійний формат
labels = np.argmax(labels, axis=1)  # Якщо мітки в one-hot кодуванні

In [5]:
# 4. Балансування вибірки
balanced_data = []
for label in np.unique(labels):
    subset = df[labels == label]
    if len(subset) < 1000:
        subset = resample(subset, replace=True, n_samples=1000, random_state=42)
    balanced_data.append(subset)

balanced_df = pd.concat(balanced_data)

# Оновлюємо токенізовані вектори та мітки
input_ids = np.stack(balanced_df['input_ids'].values)
attention_mask = np.stack(balanced_df['attention_masks'].valu=-es)
labels = np.array(balanced_df[LABEL_COLUMNS].values)
labels = np.argmax(labels, axis=1)

In [6]:
# 5. Розділення на тренувальну та тестову вибірки
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42
)
train_attention_mask, val_attention_mask = train_test_split(
    attention_mask, test_size=0.2, random_state=42
)

In [7]:
# 6. Тренування моделі
history = bert_model.fit(
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels,
    validation_data=(
        {'input_ids': val_input_ids, 'attention_mask': val_attention_mask},
        val_labels
    ),
    epochs=3,
    batch_size=32
)

Epoch 1/3
4066/4066 [==============================] - 2024s 490ms/step - loss: 0.0739 - accuracy: 0.9839 - val_loss: 0.0436 - val_accuracy: 0.9887
Epoch 2/3
4066/4066 [==============================] - 1988s 489ms/step - loss: 0.0357 - accuracy: 0.9903 - val_loss: 0.0375 - val_accuracy: 0.9887
Epoch 3/3
4066/4066 [==============================] - 1989s 489ms/step - loss: 0.0212 - accuracy: 0.9944 - val_loss: 0.0262 - val_accuracy: 0.9943


In [8]:
val_predictions = bert_model.predict({'input_ids': val_input_ids, 'attention_mask': val_attention_mask})
predicted_labels = np.argmax(val_predictions.logits, axis=1)

if len(np.unique(val_labels)) != len(LABEL_COLUMNS):
    print(f"Попередження: кількість міток у валідаційних даних ({len(np.unique(val_labels))}) не збігається з LABEL_COLUMNS.")
    LABEL_COLUMNS = LABEL_COLUMNS[:len(np.unique(val_labels))]  # Автоматичне узгодження

print("Classification Report:")
print(classification_report(val_labels, predicted_labels, target_names=LABEL_COLUMNS))

print("Confusion Matrix:")
print(confusion_matrix(val_labels, predicted_labels))

1017/1017 [==============================] - 168s 163ms/step
Попередження: кількість міток у валідаційних даних (5) не збігається з LABEL_COLUMNS.
Classification Report:
              precision    recall  f1-score   support

       toxic       1.00      1.00      1.00     31766
severe_toxic       0.88      0.47      0.61       194
     obscene       0.99      0.95      0.97       175
      threat       0.86      0.80      0.83       193
      insult       0.93      1.00      0.97       200

    accuracy                           0.99     32528
   macro avg       0.93      0.84      0.87     32528
weighted avg       0.99      0.99      0.99     32528

Confusion Matrix:
[[31732    12     1    14     7]
 [   89    91     1    11     2]
 [    8     0   167     0     0]
 [   33     1     0   154     5]
 [    0     0     0     0   200]]


In [10]:
# 7. Створення нової моделі для збереження у форматі HDF5
input_ids_layer = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask_layer = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

# Отримання виходу з BERT
bert_outputs = bert_model.bert(input_ids=input_ids_layer, attention_mask=attention_mask_layer)
logits = bert_model.classifier(bert_outputs[1])  # Використання "pooled_output"

# Створення нової функціональної моделі
new_model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=logits)

# Зберігаємо нову модель у форматі HDF5
new_model.save("model-1.h5")

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
